In [5]:
from tira.third_party_integrations import ensure_pyterrier_is_loaded, persist_and_normalize_run
from tira.rest_api_client import Client
import pyterrier as pt
from tqdm import tqdm
#from NaiveStemmer.java import NaiveStemmer

ensure_pyterrier_is_loaded()
tira = Client()

In [6]:
pt_dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')

In [7]:
class NaiveStemmer:
    def stem(self, token:str):
        cutLength = len(token) // 3
        cutLength = min(cutLength, 5)
        return token[0:len(token) - cutLength]

class CombinedStemmer:
    def stem(self, token:str):
        token = pt.index.TerrierStemmer.porter.stem(token)
        nSt = NaiveStemmer()
        return nSt.stem(token)

In [1]:
naiveStemmer = NaiveStemmer()
combinedStemmer = CombinedStemmer()
naiveStemmer = NaiveStemmer()
combinedStemmer = CombinedStemmer()
dataset = pt.get_dataset('irds:ir-lab-sose-2024/ir-acl-anthology-20240504-training')
corpus_iter = dataset.get_corpus_iter(verbose = True)
for i in corpus_iter:
    for word in i.split(' '):
        print(word)

"""
iter_indexer0 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = None , meta={'docno': 100, 'text': 4096})
indexref0 = iter_indexer0.index(tqdm(pt_dataset.get_corpus_iter()))
iter_indexer1 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = naiveStemmer , meta={'docno': 100, 'text': 4096})
indexref1 = iter_indexer1.index(tqdm(pt_dataset.get_corpus_iter()))
iter_indexer2 = pt.IterDictIndexer("./passage_index", overwrite=True, verbose= True, stemmer = combinedStemmer , meta={'docno': 100, 'text': 4096})
indexref2 = iter_indexer2.index(tqdm(pt_dataset.get_corpus_iter()))
"""

NameError: name 'NaiveStemmer' is not defined

In [ ]:
bm25NoStemmer = pt.BatchRetrieve(indexref0, wmodel="BM25")
bm25NaiveStemmer = pt.BatchRetrieve(indexref1, wmodel="BM25")
bm25CombinedStemmer = pt.BatchRetrieve(indexref2, wmodel="BM25")

In [ ]:
#pt_dataset.get_topics('text').head(3)

#print('Now we do the retrieval...')
run0 = bm25NoStemmer(pt_dataset.get_topics('text'))
run1 = bm25NaiveStemmer(pt_dataset.get_topics('text'))
run2 = bm25CombinedStemmer(pt_dataset.get_topics('text'))

#print('Done. Here are the first 10 entries of the run')
#run.head(10)

persist_and_normalize_run(run1, system_name='bm25_no_stemmer', default_output='../runs/noStemmer')
persist_and_normalize_run(run1, system_name='bm25_naive_stemmer', default_output='../runs/naiveStemmer')
persist_and_normalize_run(run2, system_name='bm25_combined_stemmer', default_output='../runs/combinedStemmer')

The run file is normalized outside the TIRA sandbox, I will store it at "../runs/noStemmer".
Done. run file is stored under "../runs/noStemmer/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs/naiveStemmer".
Done. run file is stored under "../runs/naiveStemmer/run.txt".
The run file is normalized outside the TIRA sandbox, I will store it at "../runs/combinedStemmer".
Done. run file is stored under "../runs/combinedStemmer/run.txt".
